### Import python library

In [23]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

# 1. Train, Test

In [2]:
PCT_lstm_train_X = pd.read_csv("PCT/PCT_lstm_final_train_X.csv")
PCT_lstm_train_y = pd.read_csv("PCT/PCT_lstm_final_train_y.csv")

PCT_lstm_test_X = pd.read_csv("PCT/PCT_lstm_final_test_X.csv")
PCT_lstm_test_y = pd.read_csv("PCT/PCT_lstm_final_test_y.csv")

In [3]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [4]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

#### 4) input shape로 변경 (row, timestep=2, feature)

ex) 
timestep = 2

* X_train_v 구성예시: [[1 ~ 24경기 데이터, 25 ~ 48경기 데이터], [49 ~ 72경기 데이터, 73 ~ 96경기 데이터] ]  
X_train_v.shape >> (2,2*x)             # x: 각 24group에 대한 변수 개수
* y_train_v 구성예시: 97 ~ 120 경기 승률

=> reshape

* X_train_v.shape >> (2,2,x)  # row, timestep, feature

### 2. 모델 구성(LSTM)
- optimizer: Adam -> lr(learning rate) 조절
- LSTM: 모델이 계속 동일한 결과값이 나올 때, input 뉴런 개수를 늘려야 한다는 글을 읽고 계속 input 노드 개수를 바꿔주면서 모델 생성중
- loss: MSE

- early_stop: patience를 크게하면 과적합 되는 경우가 있어서 최대한 작게 설정해둠
- batch_size: 모델이 계속 동일한 결과값이 나올 때, 데이터가 적어 batch size를 줄여보라는 글을 읽고 1로 설정해둠

In [8]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred',"shift_PCT_1","shift_PCT_2",'rms','rms0.5'])

idx = 0
for y in year:
    tmp1 = PCT_lstm_train_X[PCT_lstm_train_X["YEAR"] == y]
    tmp2 = PCT_lstm_train_y[PCT_lstm_train_y["YEAR"] == y]
    tmp3 = PCT_lstm_test_X[PCT_lstm_test_X["YEAR"] == y]
    tmp4 = PCT_lstm_test_y[PCT_lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        
        X_train_t = X_train_v.reshape(X_train_v.shape[0], 2,X_train_v.shape[1]//2)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], 2,X_test_v.shape[1]//2)
        
        ## model
        K.clear_session() 

        model = Sequential()
        optimizer = Adam(lr=0.01)

        model.add(LSTM(100,input_shape = (2,X_train_v.shape[1]//2))) # (timestep, feature)
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

        model.summary()
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train_t, y_train_v, epochs=100,
                  batch_size=1, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test_t)
        rms = sqrt(mean_squared_error(y_test_v, y_pred))
        rms05 = sqrt(mean_squared_error(y_test_v,[0.5]))
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0],
                                  X_test.loc[X_test.index[0],["shift_PCT_1"]][0],
                                  X_test.loc[X_test.index[0],["shift_PCT_2"]][0], rms,rms05]

        idx += 1

test_pred_df[['y','y_pred',"shift_PCT_1","shift_PCT_2",'rms','rms0.5']] = test_pred_df[['y','y_pred',"shift_PCT_1","shift_PCT_2",'rms','rms0.5']].astype(float)

2016LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 1s 28ms/sample - loss: 0.3020 - mae: 0.3263
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0231 - mae: 0.1438
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0270 - mae: 0.1436
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0242 - mae: 0.1363
Epoch 00004: early stopping
2016HH ===========================

49/49 [==============================] - 2s 40ms/sample - loss: 0.3008 - mae: 0.2350
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0038 - mae: 0.0517
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0034 - mae: 0.0513
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0032 - mae: 0.0473
Epoch 5/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0032 - mae: 0.0466
Epoch 6/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0027 - mae: 0.0448
Epoch 7/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0029 - mae: 0.0468
Epoch 8/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0029 - mae: 0.0458
Epoch 00008: early stopping
2016LT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)     

49/49 [==============================] - 2s 31ms/sample - loss: 0.2139 - mae: 0.2256
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0054 - mae: 0.0608
Epoch 3/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0053 - mae: 0.0594
Epoch 4/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0046 - mae: 0.0555
Epoch 5/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0046 - mae: 0.0530
Epoch 6/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0065 - mae: 0.0646
Epoch 00006: early stopping
2017HT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total para

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 1s 30ms/sample - loss: 0.2588 - mae: 0.2683
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0159 - mae: 0.0997
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0127 - mae: 0.0939
Epoch 4/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0128 - mae: 0.0906
Epoch 5/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0134 - mae: 0.0933
Epoch 00005: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 1s 29ms/sample - loss: 0.2066 - mae: 0.2284
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0028 - mae: 0.0441
Epoch 3/100
49/49 [==============================] - ETA: 0s - loss: 0.0029 - mae: 0.046 - 0s 2ms/sample - loss: 0.0029 - mae: 0.0452
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0027 - mae: 0.0415
Epoch 5/100
49/49 [==============================] - 0s 3ms/sample - lo

Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0041 - mae: 0.0549
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0042 - mae: 0.0526
Epoch 4/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0035 - mae: 0.0496
Epoch 5/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0029 - mae: 0.0470
Epoch 6/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0024 - mae: 0.0394
Epoch 7/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0024 - mae: 0.0390
Epoch 8/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0025 - mae: 0.0425
Epoch 9/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0018 - mae: 0.0344
Epoch 10/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0017 - mae: 0.0333
Epoch 11/100
49/49 [==============================] - 0s 2ms/sample - loss: 0.0013 - mae: 0.0288
Epoch 12/100
49/49 [==================

### 3. 성능평가

In [2]:
test_pred_df

In [37]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
YEAR,,,,,,
2016,0.492391,0.494809,0.504167,0.508514,0.072740,0.075725
2017,0.498370,0.488187,0.491848,0.521937,0.105915,0.076630
2018,0.507971,0.517371,0.500362,0.497480,0.061284,0.071377
2019,0.505072,0.509891,0.508152,0.495290,0.123604,0.105072


In [38]:
from sklearn.metrics import r2_score
tmp = test_pred_df.copy()
# tmp['half']= 0.5
r2_y_predict = r2_score(tmp['y'], tmp['y_pred'])
r2_y_predict

-0.43951953874474237